In [1]:
import pandas as pd
import os

# Read the CSV dataset
file_path = '../datasets/songs_with_sentiment_score.csv'
df = pd.read_csv(file_path, encoding='utf-8') # Use .head(5) if you want to limit the number of rows

df = df.fillna('')

print(str(len(df)) + ' rows')
df.head(5)

16650 rows


,id,artist,song,lyrics,img_src,sentiment_score
0,0,米津玄師,Lemon,夢ならばどれほどよかったでしょう未だにあなたのことを夢にみる忘れた物を取りに帰るように古びた...,https://m.media-amazon.com/images/I/51ZsVIMARh...,-0.669566
1,1,back number,クリスマスソング,どこかで鐘が鳴ってらしくない言葉が浮かんで寒さが心地よくてあれ なんで恋なんかしてんだろう聖...,https://m.media-amazon.com/images/I/31eRU7YYby...,-0.066641
2,2,GReeeeN,キセキ,明日、今日よりも好きになれる 溢れる想いが止まらない今もこんなに好きでいるのに 言葉に出来な...,https://m.media-amazon.com/images/I/51L0WT553N...,0.651807
3,3,back number,花束,どう思う？これから2人でやっていけると思う？んんどうかなぁでもとりあえずは一緒にいたいと思っ...,https://m.media-amazon.com/images/I/51SHzjh0dC...,-0.711921
4,4,RADWIMPS,前前前世 (movie ver.),やっと眼を覚ましたかい それなのになぜ眼も合わせやしないんだい？「遅いよ」と怒る君 これでも...,https://m.media-amazon.com/images/I/51h10DfD1o...,-0.626399


In [2]:
def generate_metadata(row):
    metadata = (
        f"{row['artist']} / "
        f"{row['song']} / "
        f"{row['lyrics']}"
    )
    return metadata

df['metadata'] = df.apply(generate_metadata, axis=1)

pd.set_option('display.max_colwidth', None)
print("A sample value from column 'metadata':")
df['metadata'].head(1)

A sample value from column 'metadata':


0    米津玄師 / Lemon / 夢ならばどれほどよかったでしょう未だにあなたのことを夢にみる忘れた物を取りに帰るように古びた思い出の埃を払う戻らない幸せがあることを最後にあなたが教えてくれた言えずに隠してた昏い過去もあなたがいなきゃ永遠に昏いままきっともうこれ以上　傷つくことなどありはしないとわかっているあの日の悲しみさえ　あの日の苦しみさえそのすべてを愛してた　あなたとともに胸に残り離れない　苦いレモンの匂い雨が降り止むまでは帰れない今でもあなたはわたしの光暗闇であなたの背をなぞったその輪郭を鮮明に覚えている受け止めきれないものと出会うたび溢れてやまないのは涙だけ何をしていたの　何を見ていたのわたしの知らない横顔でどこかであなたが今　わたしと同じ様な涙にくれ　淋しさの中にいるならわたしのことなどどうか　忘れてくださいそんなことを心から願うほどに今でもあなたはわたしの光自分が思うより恋をしていたあなたにあれから思うように息ができないあんなに側にいたのにまるで嘘みたいとても忘れられないそれだけが確かあの日の悲しみさえ　あの日の苦しみさえそのすべてを愛してた　あなたとともに胸に残り離れない　苦いレモンの匂い雨が降り止むまでは帰れない切り分けた果実の片方の様に今でもあなたはわたしの光
Name: metadata, dtype: object

In [3]:
from openai import OpenAI
import openai
from tqdm import tqdm

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
)

# Generate the vector embeddings for selected columns
def get_embedding(text, model='text-embedding-3-small'):
    try:
        # text = text.replace('\n', ' ')
        return client.embeddings.create(input = [text], model=model).data[0].embedding
    except Exception as e:
        logging.error(f'Error generating embeddings: {e}. Found issue in the following text: {text}.')
        text = 'No data available'
        return client.embeddings.create(input = [text], model=model).data[0].embedding

tqdm.pandas()
df['metadata_vector'] = df['metadata'].progress_apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
print('Conversion to vector embeddings has been completed.')

df.head(5)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16650/16650 [54:52<00:00,  5.06it/s]

Conversion to vector embeddings has been completed.


,id,artist,song,lyrics,img_src,sentiment_score,metadata,metadata_vector
0,0,米津玄師,Lemon,夢ならばどれほどよかったでしょう未だにあなたのことを夢にみる忘れた物を取りに帰るように古びた思い出の埃を払う戻らない幸せがあることを最後にあなたが教えてくれた言えずに隠してた昏い過去もあなたがいなきゃ永遠に昏いままきっともうこれ以上 傷つくことなどありはしないとわかっているあの日の悲しみさえ あの日の苦しみさえそのすべてを愛してた あなたとともに胸に残り離れない 苦いレモンの匂い雨が降り止むまでは帰れない今でもあなたはわたしの光暗闇であなたの背をなぞったその輪郭を鮮明に覚えている受け止めきれないものと出会うたび溢れてやまないのは涙だけ何をしていたの 何を見ていたのわたしの知らない横顔でどこかであなたが今 わたしと同じ様な涙にくれ 淋しさの中にいるならわたしのことなどどうか 忘れてくださいそんなことを心から願うほどに今でもあなたはわたしの光自分が思うより恋をしていたあなたにあれから思うように息ができないあんなに側にいたのにまるで嘘みたいとても忘れられないそれだけが確かあの日の悲しみさえ あの日の苦しみさえそのすべてを愛してた あなたとともに胸に残り離れない 苦いレモンの匂い雨が降り止むまでは帰れない切り分けた果実の片方の様に今でもあなたはわたしの光,https://m.media-amazon.com/images/I/51ZsVIMARhL._SL240_.jpg,-0.669566,米津玄師 / Lemon / 夢ならばどれほどよかったでしょう未だにあなたのことを夢にみる忘れた物を取りに帰るように古びた思い出の埃を払う戻らない幸せがあることを最後にあなたが教えてくれた言えずに隠してた昏い過去もあなたがいなきゃ永遠に昏いままきっともうこれ以上 傷つくことなどありはしないとわかっているあの日の悲しみさえ あの日の苦しみさえそのすべてを愛してた あなたとともに胸に残り離れない 苦いレモンの匂い雨が降り止むまでは帰れない今でもあなたはわたしの光暗闇であなたの背をなぞったその輪郭を鮮明に覚えている受け止めきれないものと出会うたび溢れてやまないのは涙だけ何をしていたの 何を見ていたのわたしの知らない横顔でどこかであなたが今 わたしと同じ様な涙にくれ 淋しさの中にいるならわたしのことなどどうか 忘れてくださいそんなことを心から願うほどに今でもあなたはわたしの光自分が思うより恋をしていたあなたにあれから思うように息ができないあんなに側にいたのにまるで嘘みたいとても忘れられないそれだけが確かあの日の悲しみさえ あの日の苦しみさえそのすべてを愛してた あなたとともに胸に残り離れない 苦いレモンの匂い雨が降り止むまでは帰れない切り分けた果実の片方の様に今でもあなたはわたしの光,"[0.0367952436208725, -0.01923556998372078, -0.06074194610118866, -0.017596915364265442, 0.06576963514089584, -0.01675896719098091, 0.035529009997844696, 0.013909942470490932, -0.027391601353883743, 0.03768905624747276, -0.0036590411327779293, 0.03068753145635128, -0.01699173077940941, -0.04599405452609062, 0.003479813225567341, -0.0011643990874290466, 0.03597591444849968, -0.009091739542782307, -0.004669234622269869, 0.013109236024320126, -0.007029455620795488, 0.05344248563051224, 0.022754952311515808, 0.004804237280040979, -0.02390945889055729, -0.04625475034117699, 0.07031317800283432, 0.027819884940981865, 0.05124519765377045, -0.03508210554718971, -0.0009118507150560617, -0.03234480693936348, 0.0030701495707035065, -0.06982903182506561, -0.011666103266179562, 0.02234528958797455, 0.031059952452778816, -0.00891483947634697, -0.019477643072605133, 0.0022356926929205656, 0.0337972491979599, -0.011135402135550976, 0.06279025971889496, 0.02132113091647625, 0.04677613824605942, 0.019403159618377686, -0.01977558061480522, 0.061747483909130096, 0.012103698216378689, 0.031227542087435722, -0.029812339693307877, 0.004711132030934095, 0.005623564589768648, 0.10174555331468582, -0.013695800676941872, -0.015064449049532413, 0.012410946190357208, 0.03141375258564949, 0.006456857547163963, 0.05228797718882561, 0.023034268990159035, -0.03254963830113411, -0.02767091616988182, 0.014012358151376247, -0.001437314203940332, 0.025846051052212715, 0.009608474560081959, -0.0178482998162508, -0.0328475758433342, 0.004329399671405554, 0.01898418553173542, 0.014254432171583176, 0.008854321204125881, -0.022717710584402084, -0.03390897810459137, -0.007294805720448494, 0.056421857327222824, 0.024672923609614372, 0.012578535825014114, -0.018127616494894028, 0.0017236132407560945, -0.007816196419298649, -0.02340668998658657, -0.02359290048480034, -0.021581824868917465, -0.021079055964946747, -0.08535900712013245, -0.022810816764831543, 0.050276901572942734, -0.000779175607021898, -0.04089187830686569, -0.02286667935550213, 0.000204540861886926, -0.001711975084617734, -0.011014365591108799, 0.018639694899320602, -0.01740139350295067, 0.01607929728925228, -0.002576691098511219, 0.0032377394381910563, ...]"
1,1,back number,クリスマスソング,どこかで鐘が鳴ってらしくない言葉が浮かんで寒さが心地よくてあれ なんで恋なんかしてんだろう聖夜だなんだと繰り返す歌とわざとらしくきらめく街のせいかな会いたいと思う回数が会えないと痛いこの胸が君の事どう思うか教えようとしてるいいよ そんな事自分で分かってるよサンタとやらに頼んでも仕方ないよなぁできれば横にいて欲しくてどこにも行って欲しくなくて僕の事だけをずっと考えていて欲しいでもこんな事を伝えたら格好悪いし長くなるだけだからまとめるよ君が好きだはしゃぐ恋人達はトナカイのツノなんか生やしてよく人前で出来るなぁいや 羨ましくなんてないけど君が喜ぶプレゼントってなんだろう僕だけがあげられるものってなんだろう大好きだと言った返事が思ってたのとは違ってもそれだけで嫌いになんてなれやしないから星に願いをなんてさ 柄じゃないけど結局君じゃないと嫌なんだって見上げてるんだあの時君に出会って ただそれだけで自分も知らなかった自分が次から次に会いたいと毎日思っててそれを君に知って欲しくてすれ違う人混みに君を探しているこんな日は他の誰かと笑ってるかな胸の奥の奥が苦しくなるできれば横にいて欲しくてどこにも行って欲しくなくて僕の事だけをずっと考えていて

In [4]:
file_path = 'songs_with_embeddings.csv'
df.to_csv(file_path, index=False, encoding='utf-8')